In [14]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

class LearnedQuantizedDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=None):
        super(LearnedQuantizedDense, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                         initializer='glorot_uniform',
                         trainable=True)
        #self.w = self.add_weight(shape=(input_shape[-1], self.units),
        #                         initializer='random_normal',
        #s                         trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='zeros',
                                 trainable=True)
        self.scale = self.add_weight(shape=(1,),
                                     initializer='ones',
                                     trainable=True)
        self.zero_point = self.add_weight(shape=(1,),
                                          initializer='zeros',
                                          trainable=True)

    def call(self, inputs):
        quantized_w = tf.stop_gradient(tf.round(self.w / self.scale + self.zero_point)) + (self.w / self.scale + self.zero_point - tf.stop_gradient(self.w / self.scale + self.zero_point))
        dequantized_w = (quantized_w - self.zero_point) * self.scale
        quantized_b = tf.stop_gradient(tf.round(self.b / self.scale + self.zero_point)) + (self.b / self.scale + self.zero_point - tf.stop_gradient(self.b / self.scale + self.zero_point))
        dequantized_b = (quantized_b - self.zero_point) * self.scale

        output = tf.matmul(inputs, dequantized_w) + dequantized_b
        if self.activation is not None:
            output = self.activation(output)
        return output

from tensorflow.keras.optimizers import Adam

# Define the model with quantized layers
input_layer = Input(shape=(28, 28, 1))
flatten_layer = Flatten()(input_layer)
quantized_dense_layer = LearnedQuantizedDense(128, activation='relu')(flatten_layer)
output_layer = LearnedQuantizedDense(10, activation='softmax')(quantized_dense_layer)

quantized_model = Model(inputs=input_layer, outputs=output_layer)

quantized_model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

quantized_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

loss, accuracy = quantized_model.evaluate(x_test, y_test)
print(f'Quantized Model Test Accuracy: {accuracy}')


Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 2.3034 - accuracy: 0.1046 - val_loss: 2.3022 - val_accuracy: 0.1135
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 2.3016 - accuracy: 0.1124 - val_loss: 2.3012 - val_accuracy: 0.1135
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 2.3014 - accuracy: 0.1124 - val_loss: 2.3011 - val_accuracy: 0.1135
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 2.3013 - accuracy: 0.1124 - val_loss: 2.3012 - val_accuracy: 0.1135
Epoch 5/5
313/313 [==============================] - 0s 936us/step - loss: 2.3010 - accuracy: 0.1135
Quantized Model Test Accuracy: 0.11349999904632568
